In [1]:
?propertynames

search: propertynames



```
propertynames(x, private=false)
```

Get a tuple or a vector of the properties (`x.property`) of an object `x`. This is typically the same as [`fieldnames(typeof(x))`](@ref), but types that overload [`getproperty`](@ref) should generally overload `propertynames` as well to get the properties of an instance of the type.

`propertynames(x)` may return only "public" property names that are part of the documented interface of `x`.   If you want it to also return "private" fieldnames intended for internal use, pass `true` for the optional second argument. REPL tab completion on `x.` shows only the `private=false` properties.


In [2]:
module O

Base.@kwdef mutable struct Foo
    pub::String = "meow"
    priv::Int = 3
end

function Base.getproperty(foo::Foo, name::Symbol)
    name === :multiple && return getfield(foo, :pub)^getfield(foo, :priv)
    hasproperty(foo, name) && return getfield(foo, name)
    error("type Foo has no public property $name")
end

Base.show(io::IO, foo::Foo) = print(io, "Foo(pub = ", repr(foo.pub), ')')

Base.propertynames(foo::Foo, private::Bool=false) =
    private ? (fieldnames(typeof(foo))..., :multiple) : (:pub, :multiple)

end

Main.O

In [3]:
foo = O.Foo()

Foo(pub = "meow")

In [4]:
propertynames(foo)

(:pub, :multiple)

In [5]:
@which propertynames(foo)

propertynames(foo::Main.O.Foo) in Main.O at In[2]:16

In [6]:
methodswith(O.Foo)

[1] getproperty(foo::Main.O.Foo, name::Symbol) in Main.O at In[2]:8
[2] propertynames(foo::Main.O.Foo) in Main.O at In[2]:16
[3] propertynames(foo::Main.O.Foo, private::Bool) in Main.O at In[2]:16
[4] show(io::IO, foo::Main.O.Foo) in Main.O at In[2]:14

In [7]:
propertynames(foo, true)

(:pub, :priv, :multiple)

In [8]:
foo.multiple

"meowmeowmeow"

In [9]:
foo.pub = "bow"
foo

Foo(pub = "bow")

In [10]:
foo.multiple

"bowbowbow"

In [11]:
foo.priv

LoadError: type Foo has no public property priv

In [12]:
foo.pub

"bow"

In [13]:
f(foo::O.Foo) = foo.multiple

f (generic function with 1 method)

In [14]:
f(foo)

"bowbowbow"

In [15]:
@code_typed f(foo)

CodeInfo(
1 ─ %1 = Main.O.getfield(foo, :pub)::String
│   %2 = Main.O.getfield(foo, :priv)::Int64
│   %3 = invoke Base.repeat(%1::String, %2::Int64)::String
└──      return %3
) => String

In [16]:
g(foo::O.Foo) = foo.pub

g (generic function with 1 method)

In [17]:
g(foo)

"bow"

In [18]:
@code_typed g(foo)

CodeInfo(
1 ─ %1 = Main.O.getfield(foo, :pub)::String
└──      return %1
) => String

In [19]:
foo.priv = 10

10

In [20]:
foo.multiple

"bowbowbowbowbowbowbowbowbowbow"